In [ ]:
import numpy as np
import sklearn
import sklearn.metrics.pairwise
import pandas as pd

def cal_alignment_acc(alignment, gt):
    gt = gt.to_numpy()

    # alignment[np.where(alignment!=np.max(alignment))] = 0
    # alignment[np.where(alignment==np.max(alignment))] = 1

    # Find the maximum value in each row
    # max_values = np.max(alignment, axis=1)
    # print(gt)
    # Create a new array with zeros and set the maximum values in each row
    result = np.zeros_like(alignment)
    for i in range(alignment.shape[0]):
        result[i, np.argmax(alignment[i])] = 1
    #     print(np.argmax(gt[i]))
    #     print(np.argmax(alignment[i]))
    
    # print(result)
    
    s = (result * gt).sum()

    acc = s / alignment.shape[1]
    return acc

def get_gt_result(alignment, gt):
    gt = gt.to_numpy()

    # alignment[np.where(alignment!=np.max(alignment))] = 0
    # alignment[np.where(alignment==np.max(alignment))] = 1

    # Find the maximum value in each row
    # max_values = np.max(alignment, axis=1)
    # print(gt)
    # Create a new array with zeros and set the maximum values in each row
    result = np.zeros_like(alignment)
    for i in range(alignment.shape[0]):
        result[i, np.argmax(alignment[i])] = 1
        # print(np.argmax(gt[i]))
        # print(np.argmax(alignment[i]))
    return gt, result

In [ ]:
import numpy as np
import sklearn
import sklearn.metrics.pairwise
import pandas as pd
import os

def cal_layer_acc(alignment, labels):
    res = []
    l_dict = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6}
    cnt0 = 0
    cnt1 = 0
    cnt2 = 0
    cnt3 = 0
    cnt4 = 0
    cnt5 = 0
    cnt6 = 0
    for i, elem in enumerate(alignment):
        if labels[i] == '-1' or labels[elem.argmax() + alignment.shape[0]] == '-1':
            continue
        if labels[i] == '-2147483648' or labels[elem.argmax() + alignment.shape[0]] == '-2147483648':
            continue
        if l_dict[labels[i]] == l_dict[labels[elem.argmax() + alignment.shape[0]]]:
            cnt0 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 1:
            cnt1 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 2:
            cnt2 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 3:
            cnt3 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 4:
            cnt4 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 5:
            cnt5 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 6:
            cnt6 += 1
    print(alignment.shape[0])
    print(cnt0/alignment.shape[0], cnt1/alignment.shape[0], cnt2/alignment.shape[0], cnt3/alignment.shape[0], cnt4/alignment.shape[0], cnt5/alignment.shape[0], cnt6/alignment.shape[0])
    res.extend([cnt0/alignment.shape[0], cnt1/alignment.shape[0], cnt2/alignment.shape[0], cnt3/alignment.shape[0], cnt4/alignment.shape[0], cnt5/alignment.shape[0], cnt6/alignment.shape[0]])
    return res

In [ ]:
def load_paste(path_='../samples/paste_simdata', sec='151673_DLPFC_151673_overlap=20%_pseudocount_1.0.h5', iter_='0'):
    alignment=np.load(os.path.join(path_, sec, "iter"+iter_+"embedding.npy"))
    labels=np.load(os.path.join(path_, sec, "iter"+iter_+"labels.npy"), allow_pickle=True)
    
    return labels, alignment

In [ ]:
# https://zenodo.org/records/10800745
# please refer to the uploaded sim data here


sec_list = ['DLPFC_151673_overlap=20%_pseudocount_1_spotIndex.h5',
 'DLPFC_151673_overlap=40%_pseudocount_1_spotIndex.h5',
 'DLPFC_151673_overlap=60%_pseudocount_1_spotIndex.h5',
 'DLPFC_151673_overlap=80%_pseudocount_1_spotIndex.h5',
 'DLPFC_151673_overlap=100%_pseudocount_1_spotIndex.h5']
ori_ = "DLPFC_151673original_spotIndex.h5"
dict_ = {'STAligner':[], 'PASTE2':[], 'PASTE':[],'PRECAST':[],'SPACEL':[],'DeepST':[], 'SPIRAL':[],'STalign':[], 'GPSA':[]}
for sec in sec_list:
    print(sec)
    gt = pd.read_csv("./" + '_overlap=' + sec.split('=')[-1].split('%')[0] +'%mapping_matrix_ground_truth_spotIndex.csv', index_col=0)
    
    # paste
    # /data/maiziezhou_lab/manfeifei/Projects/Benchmark/Test_simulatedData
    labels, alignment = load_paste(sec=ori_+"_"+sec)
    res2 = cal_alignment_acc(alignment, gt)
    dict_['PASTE'].append(res2)


plotdata = pd.DataFrame(
dict_
, 
index=['20%', '40%', '60%', '80%', '100%']
)
print(plotdata)
print()
df = pd.DataFrame(data=plotdata)

In [ ]:
cmap = ['#808000', '#000075', '#ffd8b1', '#9A6324', '#a9a9a9', '#4363d8', "#808080", "#fffac8", "#469990"]
order = ["STAligner", "PASTE2", "PASTE", "PRECAST", "SPACEL", "DeepST", "SPIRAL", "STalign", "GPSA"]

In [ ]:
df = pd.DataFrame(data=plotdata)
df_transposed = df.T

# Sort the transposed DataFrame by the values in the row corresponding to 'DLPFC151673'
df_transposed_sorted = df_transposed.sort_values(by='0.0', ascending=False)

# Transpose back to the original orientation
df_sorted = df_transposed_sorted.T

c_dict = {}
for i in range(len(order)):
    c_dict[order[i]] = cmap[i]

new_cmap = []
for i in df_sorted.columns:
    new_cmap.append(c_dict[i])

In [ ]:
import matplotlib.pyplot as plt

my_colors = new_cmap
ax = df_sorted.plot.bar(color=my_colors, width=0.8, figsize=(60, 20)) # color=['#911eb4', '#4363d8']
# plt.ylim([0, 0.85])
# ax.set_xticklabels(rotation = (45), fontsize = 13, labels=["BASS", "BASS(joint)", "BASS", "BASS(joint)", "PRECAST", "PRECAST(joint)", "PRECAST", "PRECAST(joint)"])
plt.xticks(rotation = (45))
plt.yticks(rotation = (30))
# plt.title(("Simulated data with different overlapping percentage"))
plt.xlabel('Overlapping percentage')
plt.ylabel('Spot-to-spot alignment accuracy')
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.savefig(os.path.join("./", "sim_acc_overlap_viz.pdf"), bbox_inches="tight")